# Imports

In [ ]:
import psycopg2
import psycopg2.extras

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

# Section A: Getting Data by kaggle API

## A.1 First we need to add our kaggle.jason file (extracted from kaggle account) to our computer directory.

In [ ]:
%cd 
#!mkdir ~/.kaggle
#!mv kaggle.json ~/.kaggle

## A.2 Reviewing Covid related dataset from kaggle

In [ ]:
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()
!kaggle datasets list -s "Covid"

## 3 Downloading data

In [ ]:
api.dataset_download_files('imdevskp/corona-virus-report',unzip=True)

In [ ]:
Covid_set_name="C:/Users/TUF/OneDrive - University of New Brunswick/Desktop/desctop-2023/big data/تکلیف4/country_wise_latest.csv"
data=pd.read_csv(Covid_set_name)
data.head(5)

#  Section B: Fixing data Columns for PostgreSQL queries synatx

## B.1 Replacing signs  ( `~!@#$%^&*)(-+=,.?/\|  ) with underscore (_)

In [ ]:
modfied_columns={}
for column_I in data.columns:
    new_column=column_I
    for operations in " `~!@#$%^&*)(-+=,.?/\|":
        new_column=new_column.replace(operations, "_")
    modfied_columns[column_I]=new_column
data.rename(columns = modfied_columns,inplace = True)
data.head()

## B.2 Adding Underscore (_) before number starting columns

In [ ]:
modfied_columns2={}
for column_I in data.columns:
    first_char=column_I[0]
    if first_char in "0123456789":
        first_char= "_" + first_char
        new_column=first_char + column_I[1:]     
        modfied_columns2[column_I]=new_column
data.rename(columns = modfied_columns2,inplace = True)
data.head()

## B.3 outputing processed dataset for copying into PostgreSQL

In [ ]:
data.to_csv('C:/Program Files/PostgreSQL/15/sqldata.csv',index=False)
sql_data=pd.read_csv('sqldata.csv')
sql_data.head()

# SectionC: Copying data into PosgreSQL


## C1. Building automatic query for : Create table

In [ ]:
def Create_table_query(data,Table_name,PK_index=0):
    Ptype_adjustment={"<class 'str'>":"varchar" ,
                  "<class 'numpy.int64'>":"int","<class 'int'>":"int",
                  "<class 'numpy.float64'>":"float(24)","<class 'float'>":"float(24)" }

    query=f'Create Table {Table_name} ({data.columns[PK_index]} {Ptype_adjustment[str(type(data.iloc[0,PK_index]))]} PRIMARY KEY,'
    for index,value in enumerate(data.columns):
        if index != PK_index:
            query=f'{query} {data.columns[index]} {Ptype_adjustment[str(type(data.iloc[0,index]))]},'


    new_query = list(query)
    new_query[-1] = ')'
    new_query=''.join(new_query)
    new_query=new_query +";"
    new_query
    return new_query

Create_table_query(data,"covid_set",0)

## C2. Building automatic query for : Copy table

In [ ]:
def Create_Copy_Query(Path,Table_name):
    dataset=pd.read_csv(Path)
    headings=','.join(list(dataset.columns))
    newquery2= "COPY "+ Table_name+"(" + headings +") From " + "'" +Path + "'" + "DELIMITER ','" + "CSV HEADER;"
    return newquery2
Create_Copy_Query('C:/Program Files/PostgreSQL/15/sqldata.csv',"Covid_set") 

## C.3 Copying data into PostgreSQL

In [ ]:
def Copy_to_SQL(Path,Table_name,PK_index=0):
    #Initialization
    conn = psycopg2.connect(database="GGE6505_AS04",user='postgres',
                            password="12345",host="localhost", port='5432')
    conn.autocommit = True
    cursor = conn.cursor()
    # reading data into dataframe
    dataset=pd.read_csv(Path)
    # creating table
    SQL_Create_table = Create_table_query(data,Table_name,PK_index)
    cursor.execute(SQL_Create_table)
    # Copy table
    SQL_Copy_table = Create_Copy_Query(Path,Table_name)
    cursor.execute(SQL_Copy_table)
    #Finalization
    conn.commit()
    conn.close()

Copy_to_SQL(Path='C:/Program Files/PostgreSQL/15/sqldata.csv',
            Table_name="mytbl",
            PK_index=0)


## C.4 Query_execution

In [ ]:
def Execute_Query(SQL):
    #Initialization
    conn = psycopg2.connect(database="GGE6505_AS04",user='postgres',
                            password="12345",host="localhost", port='5432')
    conn.autocommit = True
    cursor = conn.cursor()
    
    cursor.execute(SQL)
    colnames = [desc[0] for desc in cursor.description]
    df=pd.DataFrame(columns= colnames)
    for i in cursor.fetchall():
        to_append = list(i)
        df_length = len(df)
        df.loc[df_length] = to_append
          
    #Finalization
    conn.commit()
    conn.close()
    return df
    

In [ ]:
SQL1="Select country_region,deaths,Confirmed,who_region From mytbl ORDER BY deaths DESC LIMIT 4"
df_sql1=Execute_Query(SQL1)
df_sql1

In [ ]:
SQL2="Select SUM(deaths),who_region From mytbl GROUP BY who_region"
df_sql2=Execute_Query(SQL2)
plt.pie(df_sql2["sum"],  labels= df_sql2["who_region"])
plt.title(" deaths in regions")
plt.show()

# EnD